# Build a variational autoencoder

Based on the ability to generate UK car registration numbers, one can build a dataset of training and test data. With a suitable vectorizer, one can go ahead and make an autoencoder.

The decoder part of the variational encoder can then be used to generate new reg numbers.

### References

https://blog.keras.io/building-autoencoders-in-keras.html

https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import backend
from keras import optimizers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegDvlaVectorizer

### Data generation

In [2]:
n_train = 10000
n_val = 1000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegDvlaVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
val_strs = [g.get_reg() for _ in range(n_val)]
val_strs = [x for x in val_strs if x not in set(train_strs)]  # sanity check
val_vecs = np.array([v.vectorize(x) for x in val_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs) | set(val_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of validation samples = ' + str(len(val_strs)))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 150
actual number of validation samples = 1000
actual number of test samples = 1000


### Variational autoencoder

Design the architecture

In [3]:
latent_dim = 15

### Define the encoder
inputs = keras.Input(shape=(vec_length,))
h = inputs
h = layers.Dense(vec_length, activation='relu')(h)
h = layers.Dense(50, activation='relu')(h)
h = layers.Dense(25, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = backend.random_normal(shape=(backend.shape(z_mean)[0], latent_dim),
                                    mean=0., stddev=0.1)
    return z_mean + backend.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

### Define the decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = latent_inputs
x = layers.Dense(25, activation='relu')(x)
x = layers.Dense(50, activation='relu')(x)
x = layers.Dense(vec_length, activation='relu')(x)
x = layers.Dense(vec_length, activation='sigmoid')(x)
outputs = x
decoder = keras.Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

### Define the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= vec_length
kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
kl_loss = backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

Do some training

In [29]:
opt = optimizers.Adamax(learning_rate=0.004)
vae.compile(optimizer=opt, loss='binary_crossentropy')
vae.fit(train_vecs, train_vecs,
        epochs=200,
        batch_size=8,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

opt = optimizers.Adamax(learning_rate=0.001)
vae.compile(optimizer=opt, loss='binary_crossentropy')
vae.fit(train_vecs, train_vecs,
        epochs=200,
        batch_size=8,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

Epoch 1/200
1250/1250 [==============================] - 1s 1ms/step - loss: 5.2654 - val_loss: 6.3100
Epoch 2/200
1250/1250 [==============================] - 1s 917us/step - loss: 5.1368 - val_loss: 6.2133
Epoch 3/200
1250/1250 [==============================] - 1s 916us/step - loss: 5.0848 - val_loss: 6.2406
Epoch 4/200
1250/1250 [==============================] - 1s 932us/step - loss: 5.0818 - val_loss: 6.2070
Epoch 5/200
1250/1250 [==============================] - 1s 930us/step - loss: 5.0522 - val_loss: 6.1792
Epoch 6/200
1250/1250 [==============================] - 1s 932us/step - loss: 5.0364 - val_loss: 6.3538
Epoch 7/200
1250/1250 [==============================] - 1s 930us/step - loss: 5.0107 - val_loss: 6.1280
Epoch 8/200
1250/1250 [==============================] - 1s 934us/step - loss: 4.9790 - val_loss: 6.3372
Epoch 9/200
1250/1250 [==============================] - 1s 934us/step - loss: 4.9800 - val_loss: 6.1396
Epoch 10/200
1250/1250 [==============================] -

Epoch 155/200
1250/1250 [==============================] - 1s 902us/step - loss: 4.4443 - val_loss: 5.7474
Epoch 156/200
1250/1250 [==============================] - 1s 895us/step - loss: 4.4535 - val_loss: 5.6236
Epoch 157/200
1250/1250 [==============================] - 1s 902us/step - loss: 4.4463 - val_loss: 5.6123
Epoch 158/200
1250/1250 [==============================] - 1s 890us/step - loss: 4.4235 - val_loss: 5.8515
Epoch 159/200
1250/1250 [==============================] - 1s 898us/step - loss: 4.4697 - val_loss: 5.7172
Epoch 160/200
1250/1250 [==============================] - 1s 907us/step - loss: 4.4347 - val_loss: 5.6582
Epoch 161/200
1250/1250 [==============================] - 1s 903us/step - loss: 4.4376 - val_loss: 5.6362
Epoch 162/200
1250/1250 [==============================] - 1s 909us/step - loss: 4.4427 - val_loss: 5.7366
Epoch 163/200
1250/1250 [==============================] - 1s 892us/step - loss: 4.4230 - val_loss: 5.6201
Epoch 164/200
1250/1250 [============

Evaluation

In [36]:
decoded_regs = vae.predict(test_vecs)
print(test_strs[:10])
recovered = [v.recover(x) for x in decoded_regs]
print(recovered[:10])
acc = np.sum([x == y for x, y in zip(recovered, test_strs)]) / len(test_strs)
print('overall accuracy = ' + str(acc))
print('by digit accuracy:')
for i_digit in range(len(test_strs[0])):
    acc = np.sum([x[i_digit] == y[i_digit] for x, y in zip(recovered, test_strs)]) / len(test_strs)
    print('    ' + str(acc))

['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
['MY33WSZ', 'DE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
overall accuracy = 0.618
by digit accuracy:
    0.985
    0.969
    0.989
    0.986
    0.92
    0.98
    0.751


### Use decoder as generator

By directly sampling in the latent space, and running through the decoder, one effectively has a data generator.

In [46]:
[v.recover(x) for x in decoder.predict(np.random.random((5, latent_dim)))]

['EW90YYO', 'BI04YDJ', 'BM90UKB', 'RI00UDJ', 'VA94YBB']

One can do some sampling and look at some stats to see what kind of data comes out. For instance, the number of reg numbers with pair of integers below 50 and above 50

In [52]:
random_vectors = 10 * (np.random.random((1000, latent_dim)) - 0.5)
generated_regs = [v.recover(x) for x in decoder.predict(random_vectors)]
n_reg_below_50 = len({x for x in generated_regs if int(x[2:4]) < 50})
n_reg_above_50 = len({x for x in generated_regs if int(x[2:4]) >= 50})
print('reg numbers below 50 = ' + str(n_reg_below_50))
print('reg numbers above 50 = ' + str(n_reg_above_50))

reg numbers below 50 = 513
reg numbers above 50 = 487


### Avoid generating any training data

There is a chance that a random generation of data might produce one of the actual registration numbers used in training. However, any input registrations are represented as a probability distribution in the latent space; so one could require that any sample vector used in the latent space to be 'sufficiently far' down the tails of all the recorded distributions.

The distributions of the training data in the latent space are readily available:

In [53]:
# Get the distributions for each reg number in the data
all_strs = train_strs + val_strs + test_strs
all_vecs = np.array([v.vectorize(x) for x in all_strs])
means, log_sigmas, _ = encoder.predict(all_vecs)
distn_dict = {x: (y,z) for x, y, z in zip(all_strs, means, np.exp(log_sigmas))}  # dict<reg no: (mean, sigma)>
distn_dict['YK66BIQ']
# for a multivariate distribution ~ N(mean, Var) valued in the latent space, 
# mean is the first vector below, and Var is a diagonal matrix, the square root
# of the diagonal entries being given in the second vector.
# If the KL loss function has done its job, the distributions should be close to N(0,1)

(array([-0.78045595,  0.6689708 ,  0.01360171,  0.01618406,  0.86150366,
        -0.8979264 , -0.38878757, -0.7176368 , -0.7298499 , -0.5948149 ,
        -0.41195926,  0.1127018 , -0.340842  ,  0.13974576, -0.2613104 ],
       dtype=float32),
 array([0.5982543 , 0.5455288 , 0.58279663, 0.99274635, 0.5150628 ,
        0.46606475, 0.6331187 , 0.5918072 , 0.64868414, 0.41233498,
        0.6192983 , 0.9779963 , 0.9589769 , 0.6669466 , 0.55386484],
       dtype=float32))

One can access the distributions that contribute to the latent space, that represent numbers below 50:

In [57]:
distns_below_50 = {k: v for k, v in distn_dict.items() if int(k[2:4]) < 50}
print('number of distributions with number below 50 = ' + str(len(distns_below_50)))

number of distributions with number below 50 = 6103


In [68]:
random_vectors = 10 * (np.random.random((1000, latent_dim)) - 0.5)
min_mahalanobis_dists = [np.min([np.linalg.norm(np.abs(x-mu)/sigma) 
                                 for mu, sigma in distns_below_50.values()])
                         for x in random_vectors]

In [83]:
mahalanobis_threshold = 13.0
filtered_random_vectors = [x for x, y in zip(random_vectors, min_mahalanobis_dists) 
                           if y < mahalanobis_threshold]
print('number of filtered random vectors = ' + str(len(filtered_random_vectors)))

number of filtered random vectors = 110


In [82]:
filtered_generated_regs = [v.recover(x) for x in decoder.predict(np.array(filtered_random_vectors))]
filtered_n_reg_below_50 = len({x for x in filtered_generated_regs if int(x[2:4]) < 50})
print('filtered reg numbers below 50 = ' + str(filtered_n_reg_below_50))

filtered reg numbers below 50 = 61
